In [4]:
import numpy as np
import pandas as pd
import os

df = pd.read_csv('MetObjects.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448203 entries, 0 to 448202
Data columns (total 43 columns):
Object Number              448203 non-null object
Is Highlight               448203 non-null bool
Is Public Domain           448203 non-null bool
Object ID                  448203 non-null int64
Department                 448203 non-null object
Object Name                445568 non-null object
Title                      416906 non-null object
Culture                    186518 non-null object
Period                     71882 non-null object
Dynasty                    23018 non-null object
Reign                      10817 non-null object
Portfolio                  20370 non-null object
Artist Role                259909 non-null object
Artist Prefix              88928 non-null object
Artist Display Name        261111 non-null object
Artist Display Bio         224064 non-null object
Artist Suffix              10212 non-null object
Artist Alpha Sort          261088 non-null object


In [118]:
public_domain = df.loc[df['Is Public Domain'] == True]
objects = public_domain['Object Name'].value_counts()
print(objects)

Print                                                  26618
Drawing                                                14165
Fragment                                                8050
Photograph                                              7682
Piece                                                   6425
Painting                                                3537
Bowl                                                    2770
Figure                                                  2244
Vase                                                    2103
Books Prints Ornament & Architecture                    2049
Dish                                                    1647
Plate                                                   1593
Panel                                                   1323
Bottle                                                  1303
Medal                                                   1278
Ornament                                                1093
Jar                     

In [133]:
public_domain = df.loc[df['Is Public Domain'] == True]
buckets = public_domain.groupby('Object Name').filter(lambda x: len(x) >= 1000)
print(type(buckets['Link Resource']))
#buckets = df.loc[df['']]
buckets.iloc[37:]
#buckets['Link Resource'].index
#objects = public_domain['Object Name'].value_counts()

<class 'pandas.core.series.Series'>


,Object Number,Is Highlight,Is Public Domain,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,...,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository
532,14.120.12,False,True,592,American Decorative Arts,Bottle,Bottle,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Glass,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
533,14.120.13,False,True,593,American Decorative Arts,Bottle,Bottle,American,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Glass,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
534,15.79.10,False,True,594,American Decorative Arts,Bottle,Bottle,American,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Ceramics,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
537,20.109.18,False,True,597,American Decorative Arts,Bottle,Bottle,American,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Ceramics,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
538,26.277,False,True,598,American Decorative Arts,Bottle,Bottle,American,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Ceramics,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
539,30.120.297,False,True,599,American Decorative Arts,Bottle,Bottle,American,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Glass,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
541,38.78.3,False,True,601,American Decorative Arts,Bottle,Bottle,American,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Glass,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
542,"38.87.1a, b",False,True,602,American Decorative Arts,Bottle,Bottle,Chinese,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Ceramics,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
543,40.150.1,False,True,603,American Decorative Arts,Bottle,Bottle,American,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Glass,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
544,40.150.2,False,True,604,American Decorative Arts,Bottle,Bottle,American,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Glass,NaN,http://www.metmuseum.org/art/collection/search...,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"


In [71]:
public_domain['Link Resource'].index
for artwork_id, link in public_domain['Link Resource'][:3].iteritems():
    print(id, link)

34 http://www.metmuseum.org/art/collection/search/33
34 http://www.metmuseum.org/art/collection/search/34
34 http://www.metmuseum.org/art/collection/search/35


In [72]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
import re

response = requests.get(link, timeout=5)
page_content = BeautifulSoup(response.content, "html.parser")
download_button = page_content.find(class_='utility-menu__item utility-menu__item--download')

<class 'bytes'>


In [89]:
href = download_button.a['href']
img_link = re.search('https\S+.\w', href).group(0)
extension = re.search('.(\w+)$', img_link).groups()[0]
extension

'jpg'

In [59]:
response = requests.get(img_link, stream=True, timeout=5)

In [60]:
with open("images/{}.{}".format(artwork_id, extension), 'wb') as out_file:
    response.raw.decode_content = True
    shutil.copyfileobj(response.raw, out_file)
del response

NameError: name 'artwork_id' is not defined